In [ ]:
import xarray as xr
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import shapely.geometry as sgeom
import warnings
warnings.filterwarnings('ignore')
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from matplotlib.colors import LinearSegmentedColormap, BoundaryNorm

# Load Blocking_1D and AWSSI NetCDF files
ds_index = xr.open_mfdataset('/climca/people/ralawode/ATM/blocking_index/Blocking_1D_*_processed.nc', combine='by_coords')
ds_awssi = xr.open_mfdataset('/climca/people/ralawode/ATM/awssi_data/era5_awssi_*_processed.nc', combine='by_coords')

# Extract the blocking events and AWSSI total
blocking_events_all = ds_index['blocking_events']
awssi_events_all = ds_awssi['awssi_total']
awssi_temp = ds_awssi['awssi_t2m']

# Define winter season: NDJFM (November to March)
def get_winter_season(month):
    return month.isin([11, 12, 1, 2, 3])

# Extract winter data
blocking_events_winter = blocking_events_all.sel(time=get_winter_season(blocking_events_all['time.month']))
awssi_events_winter = awssi_events_all.sel(time=get_winter_season(awssi_events_all['time.month']))
awssi_temp_winter = awssi_temp.sel(time=get_winter_season(awssi_temp['time.month']))

# Compute mean for the winter season for both datasets
mean_winter_blocking = blocking_events_winter.mean(dim='time')
mean_winter_awssi = awssi_events_winter.mean(dim='time')
mean_winter_temp = awssi_temp_winter.mean(dim='time')

# Function to create arcs for regions of interest
def create_arc(lon_min, lon_max, lat_min, lat_max, n_points=100):
    lons = np.linspace(lon_min, lon_max, n_points)
    lats1 = np.full(n_points, lat_min)
    lats2 = np.full(n_points, lat_max)
    lons_combined = np.concatenate([lons, lons[::-1]])
    lats_combined = np.concatenate([lats1, lats2[::-1]])
    return sgeom.Polygon(zip(lons_combined, lats_combined))

# Create plot with two subplots
fig, axs = plt.subplots(1, 3, subplot_kw={'projection': ccrs.NorthPolarStereo()}, figsize=(10, 15))

# Create mesh grid for lat/lon
lon_grid, lat_grid = np.meshgrid(ds_index['lon'].values, ds_index['lat'].values)

# Define custom color map and normalization based on min/max AWSSI values
cmap = LinearSegmentedColormap.from_list('custom_cmap', ['white', 'gold', 'darkorange', 'red', 'darkred'])
norm_awssi = BoundaryNorm(boundaries=np.linspace(mean_winter_awssi.min().values, mean_winter_awssi.max().values, 9), ncolors=cmap.N, clip=True)
norm_temp = BoundaryNorm(boundaries=np.linspace(mean_winter_temp.min().values, mean_winter_temp.max().values, 9), ncolors=cmap.N, clip=True)
norm_blocking = BoundaryNorm(boundaries=np.linspace(0, 0.08, 9), ncolors=cmap.N, clip=True)


# Plot winter blocking events (first subplot)
ax = axs[0]
ax.add_feature(cfeature.COASTLINE, color='black')
ax.set_extent([-180, 180, 45, 90], crs=ccrs.PlateCarree())
# Transpose the blocking events data if necessary
blocking_data = mean_winter_blocking.squeeze()
if blocking_data.shape != lon_grid.shape:
    blocking_data = blocking_data.T
# Plot blocking frequency
ax.pcolormesh(lon_grid, lat_grid, blocking_data, cmap=cmap, norm=norm_blocking, transform=ccrs.PlateCarree(), shading='auto')
ax.set_title("(a) Winter Blocking Events")
# Add gridlines
gl = ax.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False
# Add arcs for regions of interest
arc_ATL = create_arc(lon_min=-16, lon_max=7.5, lat_min=47, lat_max=63)
arc_BTL = create_arc(lon_min=7.5, lon_max=40, lat_min=53, lat_max=67)
arc_PAC = create_arc(lon_min=145, lon_max=225, lat_min=64, lat_max=75)
arc_GL = create_arc(lon_min=-65, lon_max=0, lat_min=63, lat_max=75)
arc_Ural = create_arc(lon_min=45, lon_max=70, lat_min=53, lat_max=67)
ax.add_geometries([arc_ATL, arc_BTL, arc_PAC, arc_GL, arc_Ural], crs=ccrs.PlateCarree(), edgecolor='blue', facecolor='none', linewidth=2)

# Plot winter AWSSI events (second subplot)
ax = axs[1]
ax.add_feature(cfeature.COASTLINE, color='black')
ax.set_extent([-180, 180, 45, 90], crs=ccrs.PlateCarree())
# Transpose the AWSSI data if necessary
awssi_data = mean_winter_awssi.squeeze()
if awssi_data.shape != lon_grid.shape:
    awssi_data = awssi_data.T
# Plot AWSSI data
ax.pcolormesh(lon_grid, lat_grid, awssi_data, cmap=cmap, norm=norm_awssi, transform=ccrs.PlateCarree(), shading='auto')
ax.set_title("(b) Winter AWSSI Events")
# Add gridlines
gl = ax.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False


# Plot winter AWSSI events (third subplot)
ax = axs[2]
ax.add_feature(cfeature.COASTLINE, color='black')
ax.set_extent([-180, 180, 45, 90], crs=ccrs.PlateCarree())
# Transpose the AWSSI data if necessary
awssi_temp_data = mean_winter_temp.squeeze()
if awssi_temp_data.shape != lon_grid.shape:
    awssi_temp_data = awssi_temp_data.T
# Plot AWSSI data
ax.pcolormesh(lon_grid, lat_grid, awssi_temp_data, cmap=cmap, norm=norm_temp, transform=ccrs.PlateCarree(), shading='auto')
ax.set_title("(c) Winter AWSSI Temperature")
# Add gridlines
gl = ax.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False

# Adjust space between subplots
plt.subplots_adjust(wspace=0.1, hspace=0.2)

# Create a colorbar for the entire plot
cbar = fig.colorbar(ScalarMappable(norm=norm_awssi, cmap=cmap), ax=axs, orientation='horizontal', fraction=0.02, pad=0.05)
cbar.set_label('Event Frequency')

# Save the figure
output_file = '/climca/people/ralawode/ATM/winter_blocking_awssi_comparison.png'
fig.savefig(output_file, dpi=300, bbox_inches='tight')
print(f"Figure saved to {output_file}")

# Show the plot
# plt.show()


In [ ]:
# Import python packages
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore') # Suppress warnings for cleaner output

# Load the datasets
# Load all Blocking_1D and AWSSI NetCDF files and combine them by coordinates
ds_index = xr.open_mfdataset(
    '/climca/people/ralawode/ATM/blocking_index/Blocking_1D_*_processed.nc',
    combine='by_coords'
)

ds_awssi = xr.open_mfdataset(
    '/climca/people/ralawode/ATM/awssi_data/era5_awssi_*_processed.nc',
    combine='by_coords'
)

# Extract variables from the datasets
# Extract blocking events data
blocking_events_all = ds_index['blocking_events']

# Extract AWSSI total and temperature data
awssi_events_all = ds_awssi['awssi_total']
awssi_temp = ds_awssi['awssi_t2m']

# Function to assign cross-year winters (numeric winter year)
def assign_winter_season(time):
    """
    Assign a numeric winter year to each time point.
    For November and December, the winter year is the next calendar year.
    For other months, the winter year is the current year.
    """
    year = time.dt.year
    month = time.dt.month
    # If the month is November or December, increment the year by 1
    winter_year = xr.where(month >= 11, year + 1, year)
    return winter_year

# Assign winter year labels to each time point in the datasets
blocking_events_all = blocking_events_all.assign_coords(
    winter_year=assign_winter_season(blocking_events_all['time'])
)
awssi_events_all = awssi_events_all.assign_coords(
    winter_year=assign_winter_season(awssi_events_all['time'])
)
awssi_temp = awssi_temp.assign_coords(
    winter_year=assign_winter_season(awssi_temp['time'])
)

# Define regions of interest using latitude and longitude bounds
regions = {
    'ATL': {  # Atlantic region
        'lon_min': -16, 'lon_max': 7.5,
        'lat_min': 47, 'lat_max': 63
    },
    'BTL': {  # Baltic region
        'lon_min': 7.5, 'lon_max': 40,
        'lat_min': 53, 'lat_max': 67
    },
    'PAC': {  # Pacific region
        'lon_min': 145, 'lon_max': 225,
        'lat_min': 64, 'lat_max': 75
    },
    'GL': {  # Greenland region
        'lon_min': -65, 'lon_max': 0,
        'lat_min': 63, 'lat_max': 75
    },
    'Ural': {  # Ural region
        'lon_min': 45, 'lon_max': 70,
        'lat_min': 53, 'lat_max': 67
    }
}

# Function to compute winter averages for a region
def compute_region_winter_mean(data, region):
    """
    Select data within the specified region and compute the mean for each winter season.
    Args:
        data (xarray.DataArray): The data array to compute averages on.
        region (dict): Dictionary with 'lon_min', 'lon_max', 'lat_min', 'lat_max'.
    Returns:
        xarray.DataArray: Averaged data grouped by 'winter_year'.
    """
    # Select data within the latitude and longitude bounds
    regional_data = data.sel(
        lon=slice(region['lon_min'], region['lon_max']),
        lat=slice(region['lat_min'], region['lat_max'])
    )
    # Group data by 'winter_year' and compute the mean over time, latitude, and longitude
    return regional_data.groupby('winter_year').mean(dim=['time', 'lat', 'lon'])

# Compute winter averages for each region and variable
winter_averages = {}  # Dictionary to store the winter averages for each region

for region_name, region_bounds in regions.items():
    winter_averages[region_name] = {}  # Initialize dictionary for each region
    # Iterate over the variables and their corresponding data arrays
    for var_name, var_data in zip(
        ['blocking', 'awssi_total', 'awssi_temp'],
        [blocking_events_all, awssi_events_all, awssi_temp]
    ):
        # Compute the winter average for the variable in the region
        winter_avg = compute_region_winter_mean(var_data, region_bounds)
        winter_averages[region_name][var_name] = winter_avg

# Convert winter averages to pandas DataFrames for plotting
winter_averages_df = {}  # Dictionary to store DataFrames for each region

for region, data in winter_averages.items():
    # Convert each DataArray to a DataFrame
    df_blocking = data['blocking'].to_dataframe(name='Blocking Events').reset_index()
    df_awssi_total = data['awssi_total'].to_dataframe(name='AWSSI Total').reset_index()
    df_awssi_temp = data['awssi_temp'].to_dataframe(name='AWSSI Temperature').reset_index()
    
    # Merge DataFrames on 'winter_year' to ensure alignment of data
    df_merged = df_blocking.merge(df_awssi_total, on='winter_year').merge(df_awssi_temp, on='winter_year')
    
    # Store the merged DataFrame in the dictionary
    winter_averages_df[region] = df_merged

# Create subplots for each region (3 variables per region)
n_regions = len(regions)  # Number of regions
fig, axs = plt.subplots(n_regions, 3, figsize=(15, 5 * n_regions), sharex=True)

# Plot the time series for each region
for i, (region_name, df) in enumerate(winter_averages_df.items()):
    # Plot Blocking Events Time Series
    axs[i, 0].plot(
        df['winter_year'], df['Blocking Events'],
        label=f'{region_name} Blocking Events', color='blue'
    )
    axs[i, 0].set_title(f'{region_name} - Winter Blocking Events')
    axs[i, 0].set_ylabel('Blocking Events')
    axs[i, 0].tick_params(axis='x', rotation=45)  # Rotate x-axis labels for readability
    
    # Plot AWSSI Total Time Series
    axs[i, 1].plot(
        df['winter_year'], df['AWSSI Total'],
        label=f'{region_name} AWSSI Total', color='green'
    )
    axs[i, 1].set_title(f'{region_name} - Winter AWSSI Total')
    axs[i, 1].set_ylabel('AWSSI Total')
    axs[i, 1].tick_params(axis='x', rotation=45)
    
    # Plot AWSSI Temperature Time Series
    axs[i, 2].plot(
        df['winter_year'], df['AWSSI Temperature'],
        label=f'{region_name} AWSSI Temperature', color='red'
    )
    axs[i, 2].set_title(f'{region_name} - Winter AWSSI Temperature')
    axs[i, 2].set_ylabel('AWSSI Temperature (K)')  # Adjust units if necessary
    axs[i, 2].tick_params(axis='x', rotation=45)

# Set x-axis label for the bottom plots
for ax in axs[-1, :]:
    ax.set_xlabel('Winter Year')

# Adjust layout to prevent overlap and add a sup title
plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust the subplot layout
fig.suptitle('Regional Cross-Year Winter Time Series', fontsize=16)  # Add a sup title

# Save the figure to a file
output_file = '/climca/people/ralawode/ATM/regional_cross_year_winter_time_series.png'
fig.savefig(output_file, dpi=300, bbox_inches='tight')
print(f"Time series subplot figure saved to {output_file}")

# Display the plot
plt.show()
